# 预测因子的选择--三类雨型的合成分析
## 实习目的：掌握短期气候预测因子的分析和选择，加深对夏季降水分布、环流异常在短期气候预测中的物理机制的认识。
## 实习五：
- 计算1961-2010年夏季三类雨型合成图
- 计算各类雨型的前期冬季（DJF）高度场距平合成图，指出可能出现的遥相关型
## 实习六
- 计算前期冬季北太平洋海温I和II类雨型合成差值、I型和III型合成差值、II型和II型合成差值及T检验，确定关键区

北太平洋范围（120°E - 60°W，10°S-60°N）

In [1]:
# 需要用到的包
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
from cartopy.io.shapereader import Reader
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from matplotlib.ticker import MultipleLocator

plt.rcParams['font.sans-serif'] = ['SimHei']  ###防止无法显示中文并设置黑体
plt.rcParams['axes.unicode_minus'] = False  ###用来正常显示负号

In [2]:
# 读取数据
rain = xr.open_dataset('data/CN05.1_Pre_1961_2017_month_025x025.nc')['pre']
rain

<xarray.DataArray 'pre' (time: 676, lat: 163, lon: 283)>
[31183204 values with dtype=float32]
Coordinates:
  * lon      (lon) float64 69.75 70.0 70.25 70.5 ... 139.5 139.8 140.0 140.2
  * lat      (lat) float64 14.75 15.0 15.25 15.5 15.75 ... 54.5 54.75 55.0 55.25
  * time     (time) datetime64[ns] 1961-01-31 1961-02-28 ... 2017-04-30
Attributes:
    long_name:  CN05.1 daily, precipitation,mm/day                           ...

In [3]:
hgt = xr.open_dataset('data/hgt.mon.mean.nc')['hgt']
hgt

<xarray.DataArray 'hgt' (time: 855, level: 17, lat: 73, lon: 144)>
[152791920 values with dtype=float32]
Coordinates:
  * level    (level) float32 1e+03 925.0 850.0 700.0 ... 50.0 30.0 20.0 10.0
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time     (time) datetime64[ns] 1948-01-01 1948-02-01 ... 2019-03-01
Attributes:
    long_name:     Monthly mean geopotential height
    valid_range:   [ -700. 35000.]
    units:         m
    precision:     0
    GRIB_id:       7
    GRIB_name:     HGT
    var_desc:      Geopotential height
    level_desc:    Multiple levels
    statistic:     Mean
    parent_stat:   Other
    dataset:       NCEP Reanalysis Derived Products
    actual_range:  [ -354.45834 32321.098  ]

In [4]:
# 三类雨型年份
file = pd.read_csv('data/ddi-1961-2010.txt', header=None, sep='\s+')
file

,0,1,2
0,1.0,0.0,0.0
1,0.0,1.0,0.0
2,0.0,1.0,0.0
3,1.0,0.0,0.0
4,0.0,1.0,0.0
5,1.0,0.0,0.0
6,1.0,0.0,0.0
7,0.0,0.0,1.0
8,0.0,0.0,1.0
9,0.0,0.0,1.0


In [5]:
year = np.arange(1961, 2011, 1)
yearbei = []
yearz = []
yearnan = []
for i in range(len(file)):
    if ((np.array(file.loc[i]) * np.array([1, 0, 0])).sum()) == 1:
        yearbei.append(year[i])
    if ((np.array(file.loc[i]) * np.array([0, 1, 0])).sum()) == 1:
        yearz.append(year[i])
    if ((np.array(file.loc[i]) * np.array([0, 0, 1])).sum()) == 1:
        yearnan.append(year[i])

In [6]:
len(yearbei)

18

# 第一问

In [7]:
rainsum = rain.loc[rain.time.dt.month.isin([6, 7, 8])].loc['1961':'2010', :, :]  # 56年
rainsum

<xarray.DataArray 'pre' (time: 150, lat: 163, lon: 283)>
[6919350 values with dtype=float32]
Coordinates:
  * lon      (lon) float64 69.75 70.0 70.25 70.5 ... 139.5 139.8 140.0 140.2
  * lat      (lat) float64 14.75 15.0 15.25 15.5 15.75 ... 54.5 54.75 55.0 55.25
  * time     (time) datetime64[ns] 1961-06-30 1961-07-31 ... 2010-08-31
Attributes:
    long_name:  CN05.1 daily, precipitation,mm/day                           ...

In [8]:
rainsum1970_2000 = rainsum.groupby(rainsum.time.dt.year).mean(dim='time', skipna=True) * 92
rainsum1970_2000 = rainsum1970_2000.loc['1971':'2000', :, :].mean(dim='year', skipna=True)
rainsum1970_2000

<xarray.DataArray 'pre' (lat: 163, lon: 283)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)
Coordinates:
  * lon      (lon) float64 69.75 70.0 70.25 70.5 ... 139.5 139.8 140.0 140.2
  * lat      (lat) float64 14.75 15.0 15.25 15.5 15.75 ... 54.5 54.75 55.0 55.25

In [9]:
rainsum1961_2010 = rainsum.groupby(rainsum.time.dt.year).mean(dim='time', skipna=True) * 92
rainsum1961_2010

<xarray.DataArray 'pre' (year: 50, lat: 163, lon: 283)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], dtype=float32)
Coordinates:
  * lon      (lon) float64 69.75 70.0 70.25 70.5 ... 139.5 139.8 140.0 140.2
  * lat      (lat) float64 14.75 15.0 15.25 15.5 15.75 ... 54.5 54.75 55.0 55.25
  * year     (year) int64 1961 1962 1963 1964 1965 ... 2006 2007 2008 2009 2010

In [10]:
# 创建地图
def createmap(ax1, lons, lone, lats, late):
    # 海岸线
    ax1.add_geometries(Reader('D:\\maplist\\China_province\\bou2_4l.shp').geometries(), ccrs.PlateCarree(),
                       facecolor='none', edgecolor='gray', linewidth=0.8)  ###添加省界
    ax1.set_extent([lons, lone, lats, late], crs=ccrs.PlateCarree())
    # 标注坐标轴
    ax1.set_xticks(np.arange(lons, lone + 10, 5), crs=ccrs.PlateCarree())
    ax1.set_yticks(np.arange(lats, late + 10, 5), crs=ccrs.PlateCarree())
    # 设置大小刻度
    minorticks = MultipleLocator(5)
    majorticks = MultipleLocator(10)
    ax1.xaxis.set_major_locator(majorticks)
    ax1.xaxis.set_minor_locator(minorticks)
    ax1.yaxis.set_minor_locator(minorticks)
    # 经纬度格式，把0经度设置不加E和W
    lon_formatter = LongitudeFormatter(zero_direction_label=False)
    lat_formatter = LatitudeFormatter()
    ax1.xaxis.set_major_formatter(lon_formatter)
    ax1.yaxis.set_major_formatter(lat_formatter)
    ax1.grid()

# 画出三类雨型
lon=rainsum1961_2010.lon.data
lat=rainsum1961_2010.lat.data
for t_i in rainsum1961_2010.year:
    fig=plt.figure(figsize=(16,9))
    ax=fig.subplots(1,1,subplot_kw={"projection":ccrs.PlateCarree(central_longitude=105)})
    ax.set_title(str(t_i.data)+'降水距平百分率分布图')
    createmap(ax)
    colorbar=ax.contourf(lon,lat,(rainsum1961_2010.loc[t_i,:,:].data-rainsum1970_2000)/rainsum1970_2000*100,levels=[-100,-15,-5,0,10,20,100],colors=['#ed0402','#e7af2f','#e6db33','#06d08b','#04a2e2','#1d3afb'],zorder=0,extend='both',transform=ccrs.PlateCarree())
    plt.colorbar(colorbar, pad=0.05, fraction=0.04, shrink=1)

    # 保存图片
    plt.savefig('data/ex5-6/qu1/'+str(t_i.data)+'.png')
    plt.close(fig)

In [11]:
# 三类雨型
## 一类雨型
rainbei = rainsum1961_2010.loc[rainsum1961_2010.year.isin(yearbei)].mean(dim='year') - rainsum1961_2010.mean(dim='year')
## 二类雨型
rainz = rainsum1961_2010.loc[rainsum1961_2010.year.isin(yearz)].mean(dim='year') - rainsum1961_2010.mean(dim='year')
## 三类雨型
rainnan = rainsum1961_2010.loc[rainsum1961_2010.year.isin(yearnan)].mean(dim='year') - rainsum1961_2010.mean(dim='year')

In [ ]:
# t检验
def tjianyan(x1, x2, n1, n2):
    avex1 = x1.mean(axis=0)
    avex2 = x2.mean(axis=0)
    s1 = x1.std(axis=0)
    s2 = x2.std(axis=0)

    t = (avex1 - avex2) / (np.sqrt(((n1 - 1) * np.power(s1, 2) + (n2 - 1) * np.power(s2, 2)) / (n1 + n2 - 2)) * np.sqrt(1/n1+1/n2))
    return t

def tjianyan1(x1,x2,n1):
    # x1为样本，x2为总体，n1为样本量
    avex1=x1.mean(axis=0)
    avex2=x2.mean(axis=0)
    stdx1=x1.std(axis=0)
    return (avex1-avex2)/stdx1 * np.sqrt(n1)


In [ ]:
# t检验 a=0.05 n=50,t=2.009
# t_rainbei= tjianyan(rainsum1961_2010.loc[rainsum1961_2010.year.isin(yearbei)], rainsum1961_2010,18,50)
# t_rainz= tjianyan(rainsum1961_2010.loc[rainsum1961_2010.year.isin(yearz)], rainsum1961_2010,17,50)
# t_rainnan= tjianyan(rainsum1961_2010.loc[rainsum1961_2010.year.isin(yearnan)], rainsum1961_2010,15,50)
t_rainbei= tjianyan1(rainsum1961_2010.loc[rainsum1961_2010.year.isin(yearbei)], rainsum1961_2010,18)
t_rainz= tjianyan1(rainsum1961_2010.loc[rainsum1961_2010.year.isin(yearz)], rainsum1961_2010,17)
t_rainnan= tjianyan1(rainsum1961_2010.loc[rainsum1961_2010.year.isin(yearnan)], rainsum1961_2010,15)

In [ ]:
lon = rainsum1961_2010.lon.data
lat = rainsum1961_2010.lat.data
# 绘制三类雨型图
fig = plt.figure(figsize=(9, 16))
ax1, ax2, ax3 = fig.subplots(3, 1, subplot_kw={'projection': ccrs.PlateCarree(central_longitude=105)})
createmap(ax1, 69.75, 140.2, 14.75, 55.25)
createmap(ax2, 69.75, 140.2, 14.75, 55.25)
createmap(ax3, 69.75, 140.2, 14.75, 55.25)
ax1.set_title('一类雨型合成分析和t检验')
ax2.set_title('二类雨型合成分析和t检验')
ax3.set_title('三类雨型合成分析和t检验')
ax1.contourf(lon, lat, rainbei.data, levels=[-100, -15, -5, 0, 10, 20, 100],
             colors=['#ed0402', '#e7af2f', '#e6db33', '#06d08b', '#04a2e2', '#1d3afb'], zorder=0, extend='both',
             transform=ccrs.PlateCarree())
ax1.contourf(lon, lat, t_rainbei, levels=[-10, 1.734, 10], hatches=[None,'...'], zorder=1, colors="none",
             transform=ccrs.PlateCarree())
ax2.contourf(lon, lat, rainz.data, levels=[-100, -15, -5, 0, 10, 20, 100],
             colors=['#ed0402', '#e7af2f', '#e6db33', '#06d08b', '#04a2e2', '#1d3afb'], zorder=0, extend='both',
             transform=ccrs.PlateCarree())
ax2.contourf(lon, lat, t_rainz, levels=[-10, 1.740, 10], hatches=[None,'...'], zorder=1, colors="none",
             transform=ccrs.PlateCarree())
colorbar = ax3.contourf(lon, lat, rainnan.data, levels=[-100, -15, -5, 0, 10, 20, 100],
                        colors=['#ed0402', '#e7af2f', '#e6db33', '#06d08b', '#04a2e2', '#1d3afb'], zorder=0,
                        extend='both', transform=ccrs.PlateCarree())
ax3.contourf(lon, lat, t_rainnan, levels=[-10, 1.753, 10], hatches=[None,'...'], zorder=1, colors="none",
             transform=ccrs.PlateCarree())
plt.colorbar(colorbar, orientation='vertical', pad=0.05, fraction=0.04, shrink=1)
plt.colorbar(colorbar, orientation='vertical', pad=0.05, fraction=0.04, shrink=1, ax=ax2)
plt.colorbar(colorbar, orientation='vertical', pad=0.05, fraction=0.04, shrink=1, ax=ax1)

# 保存图片
plt.savefig('data/ex5-61.png',dpi=600)

In [ ]:
# 高度场合成分析
hgt500 = hgt.loc[hgt.time.dt.month.isin([12, 1, 2])].loc['1961-03-01':'2011-03-01', 500, :, :]

In [ ]:
hgt500

In [ ]:
# 把今年的12月和明年的1、2月当作今年的冬季
# 创建一个1961-2010的一维矩阵
def winsel(hgt500):
    winhgt500 = np.zeros((int(len(hgt500.time) / 3), len(hgt500.lat), len(hgt500.lon)))
    temp = np.zeros((len(hgt500.lat), len(hgt500.lon)))
    j = 0
    for i in range(len(hgt500.time)):
        temp += hgt500[i, :, :]
        if (i + 1) % 3 == 0:
            winhgt500[j, :, :] = temp / 3
            j += 1
            temp = 0
# 把win转化成 DataArray
    winhgt500 = xr.DataArray(data=winhgt500, dims=['time', 'lat', 'lon'],
                         coords={'time': pd.date_range('1961', '2011', freq='1y'),
                                 'lat': hgt500.lat.data,
                                 'lon': hgt500.lon.data})
    return winhgt500
winhgt500=winsel(hgt500)

In [ ]:
winhgt500 = winhgt500 - winhgt500.mean(dim='time')
winhgt500['time'] = np.arange(1961, 2011, 1)
winhgt500

In [ ]:
## 一类雨型hgt
hgtbei = winhgt500.loc[winhgt500.time.isin(yearbei)].mean(dim='time') - winhgt500.mean(dim='time')
## 二类雨型hgt
hgtz = winhgt500.loc[winhgt500.time.isin(yearz)].mean(dim='time') - winhgt500.mean(dim='time')
## 三类雨型hgt
hgtnan = winhgt500.loc[winhgt500.time.isin(yearnan)].mean(dim='time') - winhgt500.mean(dim='time')

In [ ]:
# t检验
thgtbei = tjianyan1(winhgt500.loc[winhgt500.time.isin(yearbei)], winhgt500, 18)
thgtz= tjianyan1(winhgt500.loc[winhgt500.time.isin(yearz)], winhgt500, 17)
thgtnan = tjianyan1(winhgt500.loc[winhgt500.time.isin(yearnan)], winhgt500, 15)

In [ ]:
# 创建地图
def createmap(ax1, lons, lone, lats, late):
    # 海岸线
    ax1.coastlines('50m')
    ax1.set_extent([lons, lone, lats, late], crs=ccrs.PlateCarree())
    # 标注坐标轴
    ax1.set_xticks(np.arange(lons, lone + 10, 30), crs=ccrs.PlateCarree())
    ax1.set_yticks(np.arange(lats, late + 10, 30), crs=ccrs.PlateCarree())
    # 设置大小刻度
    minorticks = MultipleLocator(10)
    majorticks = MultipleLocator(30)
    ax1.xaxis.set_major_locator(majorticks)
    ax1.xaxis.set_minor_locator(minorticks)
    ax1.yaxis.set_minor_locator(minorticks)
    # 经纬度格式，把0经度设置不加E和W
    lon_formatter = LongitudeFormatter(zero_direction_label=False)
    lat_formatter = LatitudeFormatter()
    ax1.xaxis.set_major_formatter(lon_formatter)
    ax1.yaxis.set_major_formatter(lat_formatter)
    ax1.grid()

In [ ]:
lon = winhgt500.lon.data
lat = winhgt500.lat.data
# 绘制三类雨型hgt距平图
fig = plt.figure(figsize=(9, 16))
ax1, ax2, ax3 = fig.subplots(3, 1, subplot_kw={'projection': ccrs.PlateCarree(central_longitude=180)})
createmap(ax1, 0, 360, -90, 90)
createmap(ax2, 0, 360, -90, 90)
createmap(ax3, 0, 360, -90, 90)
ax1.set_title('一类雨型hgt距平合成分析和t检验')
ax2.set_title('二类雨型hgt距平合成分析和t检验')
ax3.set_title('三类雨型hgt距平合成分析和t检验')
ax1.contourf(lon, lat, hgtbei.data, cmap='bwr', zorder=0, extend='both',
             transform=ccrs.PlateCarree())
ax1.contourf(lon, lat, thgtbei, levels=[-10, 1.294, 10], hatches=[None,'...'], zorder=1, colors="none",
             transform=ccrs.PlateCarree())
ax2.contourf(lon, lat, hgtz.data,cmap='bwr', zorder=0, extend='both',
             transform=ccrs.PlateCarree())
ax2.contourf(lon, lat, thgtz, levels=[-10, 1.294, 10], hatches=[None,'...'], zorder=1, colors="none",
             transform=ccrs.PlateCarree())
colorbar = ax3.contourf(lon, lat, hgtnan.data,cmap='bwr', zorder=0,
                        extend='both', transform=ccrs.PlateCarree())
ax3.contourf(lon, lat, thgtnan, levels=[-10, 1.294, 10], hatches=[None,'...'], zorder=1, colors="none",
             transform=ccrs.PlateCarree())
plt.colorbar(colorbar, orientation='vertical', pad=0.05, fraction=0.04, shrink=1)
plt.colorbar(colorbar, orientation='vertical', pad=0.05, fraction=0.04, shrink=1, ax=ax2)
plt.colorbar(colorbar, orientation='vertical', pad=0.05, fraction=0.04, shrink=1, ax=ax1)

# 保存图片
plt.savefig('data/ex5-62.png',dpi=600)

# 实习六

In [ ]:
# 读取数据
sst=xr.open_dataset('data/sst.mnmean.v4.nc')['sst'].loc['1961-03-01':'2011-03-01',:,:]
sst

In [ ]:
sst=sst.loc[sst.time.dt.month.isin([12,1,2])]
sst

In [ ]:
# 北太平洋海温（120°E~60°W，10°S~60°N）
Psst=sst.loc[:,60:-10,120:300]
Psst

In [ ]:
Psst=winsel(Psst)

In [ ]:
Psst['time']=np.arange(1961,2011,1)
Psst

In [ ]:
# 冬季北太平洋海温三类雨型      注意时间关系，比如1962年夏季降水跟1961年冬季 有关 所以得提前一年
yearbei1=list(np.array(yearbei,dtype='int64')-1)
yearz1=list(np.array(yearz,dtype='int64')-1)
yearnan1=list(np.array(yearnan,dtype='int64')-1)

In [ ]:
# 冬季北太平洋海温 I类和II类雨型 合成差值
sstbei_z=Psst.loc[Psst.time.isin(yearbei1)].mean(dim='time',skipna=True)-Psst.loc[Psst.time.isin(yearz1)].mean(dim='time',skipna=True)
# 冬季北太平洋海温 I类和III类雨型 合成差值
sstbei_nan=Psst.loc[Psst.time.isin(yearbei1)].mean(dim='time',skipna=True)-Psst.loc[Psst.time.isin(yearnan1)].mean(dim='time',skipna=True)
# 冬季北太平洋海温 II类和III类雨型 合成差值
sstz_nan=Psst.loc[Psst.time.isin(yearz1)].mean(dim='time',skipna=True)-Psst.loc[Psst.time.isin(yearnan1)].mean(dim='time',skipna=True)

In [ ]:
# t检验
t_sstbei_z=tjianyan(Psst.loc[Psst.time.isin(yearbei1)],Psst.loc[Psst.time.isin(yearz1)],18,17)
t_sstbei_nan=tjianyan(Psst.loc[Psst.time.isin(yearbei1)],Psst.loc[Psst.time.isin(yearnan1)],18,15)
t_sstz_nan=tjianyan(Psst.loc[Psst.time.isin(yearz1)],Psst.loc[Psst.time.isin(yearnan1)],17,15)

In [ ]:
sstbei_z.max()

In [ ]:
lon = Psst.lon.data
lat = Psst.lat.data
# 绘制三类雨型sst合成差值图
fig = plt.figure(figsize=(9, 16))
ax1, ax2, ax3 = fig.subplots(3, 1, subplot_kw={'projection': ccrs.PlateCarree(central_longitude=210)})
createmap(ax1, 120, 300, -10, 60)
createmap(ax2, 120, 300, -10, 60)
createmap(ax3, 120, 300, -10, 60)
ax1.set_title('I类和II类雨型sst合成差值和t检验')
ax2.set_title('I类和III类雨型sst合成差值和t检验')
ax3.set_title('II类和III类雨型sst合成差值和t检验')
ax1.contourf(lon, lat, sstbei_z.data, cmap='bwr', zorder=0, extend='both',
             transform=ccrs.PlateCarree())
ax1.contourf(lon, lat, t_sstbei_z, levels=[-10, 1.69, 10], hatches=[None,'...'], zorder=1, colors="none",
             transform=ccrs.PlateCarree())
ax2.contourf(lon, lat, sstbei_nan.data,cmap='bwr', zorder=0, extend='both',
             transform=ccrs.PlateCarree())
ax2.contourf(lon, lat, t_sstbei_nan, levels=[-10, 1.692, 10], hatches=[None,'...'], zorder=1, colors="none",
             transform=ccrs.PlateCarree())
colorbar = ax3.contourf(lon, lat, sstz_nan.data,cmap='bwr', zorder=0,
                        extend='both', transform=ccrs.PlateCarree())
ax3.contourf(lon, lat, t_sstz_nan, levels=[-10, 1.694, 10], hatches=[None,'...'], zorder=1, colors="none",
             transform=ccrs.PlateCarree())
plt.colorbar(colorbar, orientation='vertical', pad=0.05, fraction=0.04, shrink=1)
plt.colorbar(colorbar, orientation='vertical', pad=0.05, fraction=0.04, shrink=1, ax=ax2)
plt.colorbar(colorbar, orientation='vertical', pad=0.05, fraction=0.04, shrink=1, ax=ax1)

# 保存图片
plt.savefig('data/ex5-63.png',dpi=600)